<a href="https://colab.research.google.com/github/maicon-reis/financas_quantitativas/blob/main/Analisando_FIIS_com_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<hr>

## **Web Scrapping de FIIS com Python**

<hr>

In [ ]:
# Instalando as libs
!pip install -q requests
!pip install -q bs4

In [ ]:
# Bibliotecas utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import plotly.express as px

In [ ]:
# Realizando o scraping do site
fii_url = 'https://fundsexplorer.com.br/ranking'
fii_re = requests.get(fii_url)

fii_bs4 = BeautifulSoup(fii_re.text, "html.parser")
fii_table = fii_bs4.findAll(attrs={"id": "scroll-wrapper"})[0].findAll('table')
fii_df = pd.read_html(str(fii_table[0]))[0]

In [ ]:
# Criando uma lista com os tickers dos FIIS
fii_lista = fii_df['Códigodo fundo'].to_list()

print(f'Foram coletados  {len(fii_lista)} FIIS no ranking do site Funds Explorer.')

In [ ]:
# Eliminando colunas com mais de 50% de valores faltantes
fii_isna = dict(fii_df.isna().sum() / fii_df.shape[0] * 100)

for k, v in fii_isna.items():
    if fii_isna[k] > 50:
        fii_df.drop(k, axis=1, inplace=True)

In [ ]:
# Renomeando as colunas
fii_df.columns = ['codigo_fundo', 'Setor', 'preco_atual', 'liquidez_diaria',
       'dividendo', 'dividend_yield', 'dy_3m_acum', 'dy_6m_acum',
       'dy_12m_acum', 'dy_3m_media', 'dy_6m_media', 'dy_12m_media', 'DY Ano',
       'var_preco', 'Rent_periodo', 'rent_acum', 'patr_liq', 'vpa', 'p_vpa', 
       'dy_patr', 'var_patr', 'rent_patr_no_per', 'rent_patr_acum', 'qtd_ativos']

In [ ]:
# Tratamento dos dados 
fii_df = fii_df.replace(to_replace=r'^R\$ ', value='', regex=True)
fii_df = fii_df.replace(to_replace=r'%$', value='', regex=True)

fii_df['preco_atual'] = fii_df['preco_atual'].str.replace('.', '')
fii_df['patr_liq'] = fii_df['patr_liq'].str.replace('.', '')
fii_df['vpa'] = fii_df['vpa'].str.replace('.', '')

colunas = ['preco_atual', 'dividendo', 'dividend_yield', 'dy_3m_acum', 
           'dy_6m_acum', 'dy_12m_acum', 'dy_3m_media', 'dy_6m_media',
           'dy_12m_media', 'DY Ano', 'var_preco', 'Rent_periodo', 'rent_acum',
           'patr_liq', 'vpa', 'dy_patr', 'var_patr', 'rent_patr_no_per',
           'rent_patr_acum']

for coluna in colunas:
    fii_df[coluna] = fii_df[coluna].str.replace(',', '.')

In [ ]:
fii_df.info()

In [ ]:
# Modificação do tipo de dado de object para float
for coluna in colunas:
    fii_df[coluna] = fii_df[coluna].astype(float)

In [ ]:
# Excluindo os FIIS com dados faltantes.
fii_df.dropna(axis=0, inplace=True)

In [ ]:
# Resetando o índice após o tratamento dos dados
fii_df.reset_index(drop=True, inplace=True)

In [ ]:
# Verificação dos dados
fii_df.head()

In [ ]:
# Fazendo a transposição da tabela para fornecer mais uma forma de visualização.
fii_df_transposta = fii_df.transpose()

# Verificando os dados
fii_df_transposta.head()

<hr>

## **4. Análises Estatísticas exploratórias do FIIs**
<hr>

4.1. Quanto mais caro maior o DY(12m) Acumulado?

In [ ]:
fig = px.scatter(fii_df, x='preco_atual', y='dy_12m_acum', color='Setor')
fig.show()

In [ ]:
fii_df['log_preco_atual'] = np.log(fii_df['preco_atual'] + 1)

fig = px.scatter(fii_df, x='log_preco_atual', y='dy_12m_acum', color='Setor')
fig.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))

corr_matrix = fii_df.corr(method='spearman')
plot = sns.heatmap(corr_matrix, annot=True, fmt='.1f', linewidths=0.7)
plt.rcParams.update({'font.size': 10})

In [ ]:
pivot_table_FIIs = pd.pivot_table(fii_df, values='dy_12m_acum', index=['Setor'], columns=[]
                                  , aggfunc=('mean', 'median'))
pivot_table_FIIs.sort_values('mean', ascending=False)

In [ ]:
fii_df['var_patr'] = fii_df['var_patr'] < 100
fii_VPA = fii_df[fii_df['var_patr']]
fii_VPA

In [ ]:
# Dividen Yield Maior que 10%
fii_VPA_Select = fii_VPA[fii_VPA['dy_12m_acum'] > 12.75][fii_VPA['liquidez_diaria'] > fii_VPA['liquidez_diaria'].median()]
fii_VPA_Select